## Computing convergence rates
Author: Jørgen S. Dokken, Hans Petter Langtangen, Anders Logg

For any numerical method one of the most central questions is its *convergence rate*: How fast does the error go to zero when the resolution is increased (mesh size decreased).

For the finite element method, this usually corresponds to proving, theoretically or imperically, that the error $e=u_e-u_h$ is bounded by the mesh size $h$ to some power $r$, that is $\vert\vert e \vert\vert\leq Ch^r$ for some constant $C$. The number $r$ is called the *convergence rate* of the method. Note that the different norms like the $L^2$-norm $\vert\vert e\vert\vert$ or the $H_0^1$-norm have different convergence rates.

### Computing error norms
We start by creating a manufactured problem, using the same problem as in [the solver configuration](./solvers.ipynb).


In [1]:
import dolfinx
import numpy
import ufl
from mpi4py import MPI
from petsc4py import PETSc

def u_ex(mod):
    return lambda x: mod.cos(2*mod.pi*x[0])*mod.cos(2*mod.pi*x[1])

u_numpy = u_ex(numpy)
u_ufl = u_ex(ufl)

def solve_poisson(N=10, degree=1):

    mesh = dolfinx.UnitSquareMesh(MPI.COMM_WORLD, N, N)
    x = ufl.SpatialCoordinate(mesh)
    f = -ufl.div(ufl.grad(u_ufl(x)))
    V = dolfinx.FunctionSpace(mesh, ("CG", degree))
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    L = f * v * ufl.dx
    u_bc = dolfinx.Function(V)
    u_bc.interpolate(u_numpy)
    u_bc.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim -1, lambda x: numpy.full(x.shape[1], True))
    dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim-1, facets)
    bcs = [dolfinx.DirichletBC(u_bc, dofs)]
    default_problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    return default_problem.solve(), u_ufl(x)

Now, we can compute the error between the analyical solution `u_ex=u_ufl(x)` and approximated solution `uh`. A natural choice might seem to compute `(u_ex-uh)**2*ufl.dx`. 

In [2]:
import numpy as np
uh, u_ex = solve_poisson(10)
error = (uh - u_ex)**2 * ufl.dx
E = np.sqrt(dolfinx.fem.assemble_scalar(error))
print("L2-error: {0:.2e}".format(E))

L2-error: 5.28e-02


Sometimes it is of interest to compute the error fo the gradient field, $\vert\vert \nabla(u_e-u_h)\vert\vert$, often referred to as the $H_0^1$-nrom of the error, this can be expressed as

In [3]:
eh = uh - u_ex
error_H10 = ufl.dot(ufl.grad(eh), ufl.grad(eh)) * ufl.dx
E_H10 = np.sqrt(dolfinx.fem.assemble_scalar(error_H10))
print(r"H01-error: {0:.2e}".format(E_H10))

H01-error: 1.36e+00


### Computing convergence rates
Let us consider a sequence of mesh resolutions $h_0>h_1>h_2$, where $h_i=\frac{1}{N_i}$ we compute the errors for a range of $N_i$s

In [4]:
Ns = [4, 8, 16, 32, 64]
Es = np.zeros(len(Ns), dtype=PETSc.ScalarType)
hs = np.zeros(len(Ns), dtype=np.float64)
for i, N in enumerate(Ns):
    uh, u_ex = solve_poisson(N, degree=1)
    error =(uh - u_ex)**2 * ufl.dx
    Es[i] = np.sqrt(dolfinx.fem.assemble_scalar(error))
    hs[i] = 1./Ns[i]
    print("h: {0:.2e} Error: {1:.2e}".format(hs[i], Es[i]))

h: 2.50e-01 Error: 2.43e-01
h: 1.25e-01 Error: 7.96e-02
h: 6.25e-02 Error: 2.15e-02
h: 3.12e-02 Error: 5.47e-03
h: 1.56e-02 Error: 1.37e-03


If we assume that $E_i$ is of the form $E_i=Ch_i^r$, with unknown constants $C$ and $r$, we can compare two consecqutive experiments, $E_{i-1}= Ch_{i-1}^r$ and $E_i=Ch_i^r$, and solve for $r$:
```{math}
r=\frac{\ln(E_i/E_{i-1})}{\ln(h_i/h_{i-1})}
```
The $r$ values should approac the expected convergence rate (which is typically the polynomial degree + 1 for the $L^2$-error.) as $i$ increases. This can be written compactly using `numpy`.

In [5]:
rates = np.log(Es[1:]/Es[:-1])/np.log(hs[1:]/hs[:-1])
print("Rates", rates)

Rates [1.61185359 1.89145161 1.97191103 1.99290373]


We also do a similar study for different orders of polynomial spaces to verify our previous claim.

In [6]:
Ns = [4, 8, 16, 32, 64]
for degree in [1, 2, 3]:
    Es = np.zeros(len(Ns), dtype=PETSc.ScalarType)
    hs = np.zeros(len(Ns), dtype=np.float64)
    for i, N in enumerate(Ns):
        uh, u_ex = solve_poisson(N, degree=degree)
        error =(uh - u_ex)**2 * ufl.dx
        Es[i] = np.sqrt(dolfinx.fem.assemble_scalar(error))
        hs[i] = 1. / Ns[i]
        print("h: {0:.2e} Error: {1:.2e}".format(hs[i], Es[i]))
    rates = np.log(Es[1:] / Es[:-1]) / np.log(hs[1:] / hs[:-1])
    print("Polynomial degree{0:d}, Rates".format(degree), rates)

h: 2.50e-01 Error: 2.43e-01
h: 1.25e-01 Error: 7.96e-02
h: 6.25e-02 Error: 2.15e-02
h: 3.12e-02 Error: 5.47e-03
h: 1.56e-02 Error: 1.37e-03
Polynomial degree1, Rates [1.61185359 1.89145161 1.97191103 1.99290373]
h: 2.50e-01 Error: 3.52e-02
h: 1.25e-01 Error: 4.39e-03
h: 6.25e-02 Error: 5.50e-04
h: 3.12e-02 Error: 6.88e-05
h: 1.56e-02 Error: 8.60e-06
Polynomial degree2, Rates [3.00111381 2.99827832 2.99822257 2.99930784]
h: 2.50e-01 Error: 5.47e-03
h: 1.25e-01 Error: 3.33e-04
h: 6.25e-02 Error: 1.99e-05
h: 3.12e-02 Error: 1.21e-06
h: 1.56e-02 Error: 7.51e-08
Polynomial degree3, Rates [4.03741264 4.06719372 4.03410807 4.01498978]


### To be implemented: interpolation into higher order function spaces
However, as this gets expanded to `u_ex**2 + uh**2 - 2*u_ex*uh`. If the error is small, (and the solution itself is of moderate size), this calculation will correspond to subtract two positive numbers `u_ex**2 + uh**2`$\sim 1$ and `2*u_ex*u`$\sim 1$ yielding a small number, prone to round-off errors.

We start by interpolating the analytical solution and the approximated solution into a higher order function space.

### Add implementation + infinity norm estimates